In [1]:
# append path info to access api
import os, sys
#from git_root import git_root
#sys.path.append(os.path.abspath(git_root("nmdc_runtime/api/core")))
#sys.path.append(os.path.abspath(git_root("nmdc_runtime/site")))

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import json
import copy
import pandas as pd
from toolz.dicttoolz import assoc_in, merge
from pprint import pprint
from dotenv import dotenv_values
from pymongo import MongoClient

from nmdc_runtime.api.core.metadata import load_changesheet, update_mongo_db
from linkml_runtime.utils.schemaview import SchemaView

### load mongodb via env info

In [4]:
config = dotenv_values("../../.env.localhost")
config["MONGO_HOST"]

'mongodb://mongo:27018'

In [5]:
client = MongoClient(host=config["MONGO_HOST"], username=config["MONGO_USERNAME"], password=config["MONGO_PASSWORD"])
mongodb = client["nmdc_etl_staging"]

### ensure indexes on mongodb collections

In [9]:
from nmdc_runtime.util import nmdc_jsonschema, nmdc_jsonschema_validate

collection_names = {
    name for name, spec in nmdc_jsonschema["properties"].items()
    if name.endswith("_set")
}

# this causes the notebook to hang, so commenting out
# for name in collection_names:
#     mongodb[name].create_index("id", unique=True)

### load change sheet

In [10]:
pd.set_option("display.max_columns", None)
pd.set_option('display.width', 1000)

In [11]:
pd.read_csv("data/changesheet-without-separator3.tsv", sep="\t")

,id,action,attribute,value
0,gold:Gs0110138,update,name,soil study
1,NaN,NaN,ecosystem,soil
2,NaN,update,doi,v1
3,v1,NaN,has_raw_value,10.9999/8888
4,NaN,update,principal_investigator,v2
5,v2,NaN,name,v3
6,v3,NaN,first,foo
7,v3,NaN,last,bar
8,v2,NaN,has_raw_value,foo bar
9,NaN,update,description,short description


In [14]:
sheetDf = load_changesheet('data/changesheet-without-separator3.tsv')
sheetDf

,id,action,attribute,value,group_id,group_var,path
0,gold:Gs0110138,update,name,soil study,gold:Gs0110138,,name
1,,update,ecosystem,soil,gold:Gs0110138,,ecosystem
2,,update,doi,v1,gold:Gs0110138,,
3,v1,update,has_raw_value,10.9999/8888,gold:Gs0110138,v1,doi/has_raw_value
4,,update,principal_investigator,v2,gold:Gs0110138,,
5,v2,update,name,v3,gold:Gs0110138,v2,
6,v3,update,first,foo,gold:Gs0110138,v3,principal_investigator/name/first
7,v3,update,last,bar,gold:Gs0110138,v3,principal_investigator/name/last
8,v2,update,has_raw_value,foo bar,gold:Gs0110138,v2,principal_investigator/has_raw_value
9,,update,description,short description,gold:Gs0110138,,description


### update mongodb using change sheet

In [13]:
update_mongo_db(sheetDf, print_update=True, print_data=True, print_query=True)

{'_id': ObjectId('61258954a53f2364ac7723ef'), 'id': 'gold:Gs0110138', 'name': 'soil study', 'description': 'short description', 'ecosystem': 'soil', 'ecosystem_category': 'Aquatic', 'ecosystem_type': 'Freshwater', 'ecosystem_subtype': 'Wetlands', 'specific_ecosystem': 'Unclassified', 'principal_investigator': {'name': {'last': 'bar', 'first': 'foo'}, 'has_raw_value': 'foo bar'}, 'type': 'nmdc:Study', 'doi': {'has_raw_value': '10.9999/8888'}}
{
  "update": "gold.study_set",
  "updates": [
    {
      "q": {
        "id": "gold:Gs0110138"
      },
      "u": {
        "$set": {
          "name": "soil study"
        }
      }
    },
    {
      "q": {
        "id": "gold:Gs0110138"
      },
      "u": {
        "$set": {
          "ecosystem": "soil"
        }
      }
    },
    {
      "q": {
        "id": "gold:Gs0110138"
      },
      "u": {
        "$set": {
          "description": "short description"
        }
      }
    },
    {
      "q": {
        "id": "gold:Gs0110138"
      

{'n': 7, 'nModified': 0, 'ok': 1.0}